In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [2]:
fake = pd.read_csv('Datasets\Fake.csv')
true = pd.read_csv('Datasets\True.csv')

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Pankaj\AppData\Local\Temp\ipykernel_1320\3296890919.py:1: SyntaxWarning: invalid escape sequence '\F'
  fake = pd.read_csv('Datasets\Fake.csv')
C:\Users\Pankaj\AppData\Local\Temp\ipykernel_1320\3296890919.py:2: SyntaxWarning: invalid escape sequence '\T'
  true = pd.read_csv('Datasets\True.csv')


In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class'] = 0
true['class'] = 1

In [6]:
df = pd.concat([fake, true]).sample(frac=1, random_state=42)

In [7]:
# Combine title + text into one column
df['content'] = df['title'].fillna('') + " " + df['text'].fillna('') + " " + df['subject'].fillna('')

In [8]:
df.sample(10)

,title,text,subject,date,class,content
15365,DID BILL CLINTON ADVISE DONALD TRUMP TO RUN FO...,"Well, well, well the left is trying pretty muc...",politics,"Aug 6, 2015",0,DID BILL CLINTON ADVISE DONALD TRUMP TO RUN FO...
10840,Loser! Jerk! Insults fly on Republican campaig...,WASHINGTON (Reuters) - For months Republican p...,politicsNews,"February 13, 2016",1,Loser! Jerk! Insults fly on Republican campaig...
17837,Liberians vote to bolster peaceful democracy i...,MONROVIA (Reuters) - Liberians voted on Tuesda...,worldnews,"October 10, 2017",1,Liberians vote to bolster peaceful democracy i...
17994,Segregation fans fears of fresh 'cleansing' in...,"MYEBON, Myanmar (Reuters) - Buddhist villagers...",worldnews,"October 9, 2017",1,Segregation fans fears of fresh 'cleansing' in...
3828,Unhinged Ann Coulter Blames ‘Fat Girls’ For T...,While this election was undeniably (from one s...,News,"November 12, 2016",0,Unhinged Ann Coulter Blames ‘Fat Girls’ For T...
6961,North Carolina Passes Anti-LGBT Law So This P...,North Carolina is facing even more backlash fo...,News,"April 11, 2016",0,North Carolina Passes Anti-LGBT Law So This P...
7683,"More Republicans expect Clinton, rather than T...",NEW YORK (Reuters) - More Republicans now thin...,politicsNews,"October 26, 2016",1,"More Republicans expect Clinton, rather than T..."
20716,MOMS WHOSE CHILDREN Were Killed By Illegals To...,If you haven t yet discovered The Remembrance ...,left-news,"Apr 19, 2016",0,MOMS WHOSE CHILDREN Were Killed By Illegals To...
10000,Classic! Kid Rock Hits Back At Fake News And L...,Not much to say after this classic response fr...,politics,"Sep 2, 2017",0,Classic! Kid Rock Hits Back At Fake News And L...
19042,Couple defy Hurricane Maria on roof to save pe...,"YAUCO, Puerto Rico (Reuters) - Hurricane Maria...",worldnews,"September 26, 2017",1,Couple defy Hurricane Maria on roof to save pe...


In [9]:
df = df.drop(["title","subject","date","text"], axis = 1)

In [10]:
df.head()

,class,content
22216,0,Ben Stein Calls Out 9th Circuit Court: Committ...
4436,1,Trump drops Steve Bannon from National Securit...
1526,1,Puerto Rico expects U.S. to lift Jones Act shi...
1377,0,OOPS: Trump Just Accidentally Confirmed He Le...
8995,1,Donald Trump heads for Scotland to reopen a go...


In [ ]:
df.reset_index(inplace=True)

In [25]:
# help(df['content'].apply)

In [22]:
# --- Text Cleaning ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [23]:
df['cleaned'] = df['content'].apply(clean_text)

In [24]:
df.sample(10)

,level_0,index,class,content,cleaned
1533,1533,5976,1,Trump says he wants to cut Navy submarines' pr...,trump says he wants to cut navy submarines pri...
14278,14278,20479,1,Merkel and the refugees: How German leader eme...,merkel and the refugees how german leader emer...
25168,25168,11142,0,HYSTERICAL! LIBERAL TABLOID Slammed After Stag...,hysterical liberal tabloid slammed after stage...
10256,10256,2518,0,"Trump Whines About Fake News, CNN’s Chris Cuo...",trump whines about fake news cnn’s chris cuomo...
37010,37010,5813,1,A barb at Germany puts Trump administration on...,a barb at germany puts trump administration on...
17436,17436,19663,1,Jihadists launch big attack on Syrian governme...,jihadists launch big attack on syrian governme...
42914,42914,12162,1,Man with knife shot at Amsterdam airport; susp...,man with knife shot at amsterdam airport suspe...
18003,18003,7740,0,GOP Poll Workers Accused Of Cheating Dem Vote...,gop poll workers accused of cheating dem voter...
7417,7417,21805,0,BREAKING: LEFTIST DEMOCRAT MAYOR ORDERED BALTI...,breaking leftist democrat mayor ordered baltim...
7514,7514,21203,0,WATCH: PARENTS JAM MEETING ROOM To Denounce Fa...,watch parents jam meeting room to denounce fal...


In [14]:
# --- Text Cleaning ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub(r"[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\w*\d\w*", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [26]:
x =df["cleaned"]
y=df["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25, random_state=42)

In [27]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [28]:
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(xv_train, ytrain)

RandomForestClassifier(n_estimators=200, random_state=42)

In [29]:
prediction = model.predict(xv_test)
model.score(xv_test,ytest)

0.9931403118040089

In [21]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5863
           1       0.99      0.99      0.99      5362

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [30]:
joblib.dump(vectorizer,"vectorizer_rf.jb")
joblib.dump(model, "rf_model.jb")

['rf_model.jb']